# SIGMA \begin{equation*} \frac{\partial V}{\partial t} = \Psi + \Omega \end{equation*}
## **SODA 3.4.2**

In [1]:
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
cluster.scale(20)
#cluster.adapt(minimum=2, maximum=10)  # or cluster.scale(n) to a fixed size.
client = cluster.get_client()

In [3]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.7bdc6a8f773844f9a1a399ddd27399ba Dashboard: /services/dask-gateway/clusters/prod.7bdc6a8f773844f9a1a399ddd27399ba/status,Cluster Workers: 1 Cores: 2 Memory: 8.59 GB


In [1]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import fsspec
%matplotlib inline
xr.set_options(display_style='text');

In [2]:
url_ocean = 'gs://pangeo-forge-us-central1/pangeo-forge/soda/soda3.4.2_5dy_ocean_or'
fs = gcsfs.GCSFileSystem()
ocean = xr.open_zarr(fs.get_mapper(url_ocean), consolidated=True)

In [10]:
path = 'pangeo-tmp/stb2145/SODA/datasets/'

transport = xr.open_zarr(gcsfs.GCSMap(path + 'transport_wg.zarr'))
cells = xr.open_zarr(gcsfs.GCSMap(path + 'cells_wg.zarr'))
surf = xr.open_zarr(gcsfs.GCSMap(path + 'surf_terms.zarr'))
totend = xr.open_zarr(gcsfs.GCSMap(path + 'totend.zarr'))

In [11]:
transport

<xarray.Dataset>
Dimensions:         (nv: 2, st_edges_ocean: 51, st_ocean: 50, time: 985, xt_ocean: 381, xu_ocean: 381, yt_ocean: 169, yu_ocean: 169)
Coordinates:
  * nv              (nv) float64 1.0 2.0
  * st_edges_ocean  (st_edges_ocean) float64 0.0 10.07 ... 5.29e+03 5.5e+03
  * st_ocean        (st_ocean) float64 5.034 15.1 25.22 ... 5.185e+03 5.395e+03
  * time            (time) object 1993-01-07 01:00:00 ... 2019-12-17 01:00:00
  * xt_ocean        (xt_ocean) float64 -65.12 -64.88 -64.62 ... 29.62 29.88
  * xu_ocean        (xu_ocean) float64 -65.0 -64.75 -64.5 ... 29.5 29.75 30.0
  * yt_ocean        (yt_ocean) float64 -80.02 -79.92 -79.81 ... -62.25 -62.14
  * yu_ocean        (yu_ocean) float64 -79.97 -79.86 -79.76 ... -62.2 -62.08
Data variables:
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_T1      (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_T2      (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    time_bounds     (time, nv) int64 dask.array<chunksize=(1, 2), meta=np.ndarray>
    tx_int          (time, yt_ocean, xu_ocean) float32 dask.array<chunksize=(1, 169, 381), meta=np.ndarray>
    tx_trans        (time, st_ocean, yt_ocean, xu_ocean) float32 dask.array<chunksize=(1, 50, 169, 381), meta=np.ndarray>
    ty_int          (time, yu_ocean, xt_ocean) float32 dask.array<chunksize=(1, 169, 381), meta=np.ndarray>
    ty_trans        (time, st_ocean, yu_ocean, xt_ocean) float32 dask.array<chunksize=(1, 50, 169, 381), meta=np.ndarray>
Attributes:
    filename:   soda3.4.2_10dy_transport_or_1993_01_07.nc
    grid_tile:  1
    grid_type:  mosaic
    title:      MOM5_SODA_3.4.2

## <font color='Green'>Slice to WG region</font>

In [3]:
#WG boundaries (80˚S, 62˚S, 65˚W, 30˚E)
lower_lat = 0 #-80.02101494
upper_lat = 169 #-62.01972704
left_lon =  859 #-65.12
right_lon = 1240 #30.12

lower_lat_u = 0 #-79.96821565
upper_lat_u = 169 #-61.96096874 python is right exclusive so will have -62.0 in array
left_lon_u = 859 #-65.0
right_lon_u = 1240 #30.25 python is right exclusive so will have 30.0˚E

In [4]:
#set the boundaries
lat_range = dict(yt_ocean=slice(lower_lat, upper_lat), yu_ocean=slice(lower_lat_u, upper_lat_u))
lon_range = dict(xt_ocean=slice(left_lon, right_lon), xu_ocean=slice(left_lon_u, right_lon_u))

lat_range_no_u = dict(yt_ocean=slice(lower_lat, upper_lat))
lon_range_no_u = dict(xt_ocean=slice(left_lon, right_lon))

lat_range_u = dict(yu_ocean=slice(lower_lat_u, upper_lat_u))
lon_range_u = dict(xu_ocean=slice(left_lon_u, right_lon_u))

In [7]:
ds_wg = ocean.isel(**lat_range, **lon_range)

In [8]:
import xgcm

grid_wg = xgcm.Grid(ds_wg, coords={'X': {'center': 'xt_ocean', 'right': 'xu_ocean'}, 
                                      'Y': {'center': 'yt_ocean', 'right': 'yu_ocean'},
                                      'T': {'center': 'time'},
                                      'Z': {'center': 'st_edges_ocean', 'right': 'st_ocean'},
                                      'Zu': {'center': 'sw_edges_ocean', 'right': 'sw_ocean'}},
                     periodic=['X'])
grid_wg

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   xt_ocean --> right
  * right    xu_ocean --> center
Y Axis (not periodic, boundary=None):
  * center   yt_ocean --> right
  * right    yu_ocean --> center
T Axis (not periodic, boundary=None):
  * center   time
Z Axis (not periodic, boundary=None):
  * center   st_edges_ocean --> right
  * right    st_ocean --> center
Zu Axis (not periodic, boundary=None):
  * center   sw_edges_ocean --> right
  * right    sw_ocean --> center

In [ ]:
#resample ocean ds to have same time stamp as transport ds
ocean_resample = ds_wg.resample(time='10D').mean()